In [ ]:
!pip install datasets py7zr transformers evaluate rouge_score accelerate wandb

In [ ]:
from datasets import load_dataset
dataset = load_dataset('samsum')

In [ ]:
print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

In [ ]:
from random import randrange        
sample = dataset['train'][randrange(len(dataset["train"]))]
print(f"dialogue: \n{sample['dialogue']}\n---------------")
print(f"summary: \n{sample['summary']}\n---------------")

In [ ]:
from transformers import AutoTokenizer, BartForConditionalGeneration, T5ForConditionalGeneration
model_str = 'sshleifer/distilbart-cnn-6-6'
tokenizer = AutoTokenizer.from_pretrained(model_str)
model = BartForConditionalGeneration.from_pretrained(model_str)
# model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", device_map="auto")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from datasets import concatenate_datasets

tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["dialogue"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

In [ ]:
def preprocess_function(sample,padding="max_length"):
    inputs = ["summarize: " + item for item in sample["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=512, padding=padding, truncation=True)

    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
rouge = evaluate.load("rouge")
google_bleu = evaluate.load("google_bleu")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    metric_dict = {}
    rogue_score = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    gleu_score = google_bleu.compute(predictions=decoded_preds, references=decoded_labels)
    rogue_score = {k: round(v * 100, 4) for k, v in rogue_score.items()}
    gleu_score = {k: round(v * 100, 4) for k, v in gleu_score.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    metric_dict["gen_len"] = np.mean(prediction_lens)
    for k,v in rogue_score.items():
        metric_dict[k] = v
    for k,v in gleu_score.items():
        metric_dict[k] = v
    return metric_dict

In [ ]:
import json
def save_outputs(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    with open(f"/content/drive/MyDrive/ECE285-Project/preds_{model_str}.json",'w') as f:
        json.dump(decoded_preds,f)

    metric_dict = {}
    rogue_score = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    gleu_score = google_bleu.compute(predictions=decoded_preds, references=decoded_labels)
    rogue_score = {k: round(v * 100, 4) for k, v in rogue_score.items()}
    gleu_score = {k: round(v * 100, 4) for k, v in gleu_score.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    metric_dict["gen_len"] = np.mean(prediction_lens)
    for k,v in rogue_score.items():
        metric_dict[k] = v
    for k,v in gleu_score.items():
        metric_dict[k] = v
    return metric_dict

In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=4
)

In [ ]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"/content/drive/MyDrive/ECE285-Project/{model_str}"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=5,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="wandb",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()
# trainer.evaluate()
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained('BART_SAMSUM')

In [ ]:
# trainer.evaluate()

In [ ]:
# # model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/ECE285-Project/BART_SAMSUM/checkpoint-921", device_map="auto")
model = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/ECE285-Project/sshleifer/distilbart-cnn-6-6/checkpoint-3684",device_map="auto")

test_args = Seq2SeqTrainingArguments(
    output_dir='BART_SAMSUM',
    do_train=False,
    do_predict=True,
    per_device_eval_batch_size=8,
    predict_with_generate=True)


trainer = Seq2SeqTrainer(
            model=model,
            args=training_args,
            data_collator=data_collator,
            train_dataset=tokenized_dataset["train"],
            eval_dataset=tokenized_dataset["test"],
            compute_metrics = save_outputs)

trainer.evaluate()

In [ ]:
# preds_file = '/content/drive/MyDrive/ECE285-Project/preds.json'
# with open(preds_file,'r') as f:
#     preds = json.load(f)

In [ ]:
# dataset['test'][0]['dialogue']

In [ ]:
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xxl")
# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xxl")

# conversation = dataset['test'][0]['dialogue']
# summary = preds[0]
# prompt =f'Given is a conversation {conversation} and its corresponding summary {summary}, please improve upon the summary'

# inputs = tokenizer("", return_tensors="pt")
# outputs = model.generate(**inputs)
# print(tokenizer.batch_decode(outputs, skip_special_tokens=True))